<a href="https://colab.research.google.com/github/Sriram-Soundar/MatchFlow.AI/blob/main/MatchFlow_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-c

In [13]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
from google.colab import files
uploaded = files.upload()

In [8]:
import pandas as pd
df = pd.read_csv("recaps.csv")
df.head()

,input_text,target_text
0,Recap this game: Match: Vassar College Men 19 ...,When the Brewers did get the ball they were un...
1,Recap this game: Match: Vassar College Women 3...,At fullback Jude Robinson had a couple of biza...
2,Recap this game: Match: Vassar College Women 3...,Vassar lost Ella Maniatis just after the halft...
3,Recap this game: Match: Vassar College Men 22 ...,Vassar was guilty of pushing the ball wide too...
4,Recap this game: Match: Vassar College Men 53 ...,The other major factor was that the Pirates we...


**Formating the dataset for training**

In [15]:
from datasets import Dataset

# Rename columns just in case and convert to Hugging Face Dataset
df = df[['input_text', 'target_text']].dropna()
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 24
    })
    test: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 3
    })
})

**Load and tokenize using T5**

In [16]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess(example):
    model_input = tokenizer(example['input_text'], padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(example['target_text'], padding="max_length", truncation=True, max_length=512)
    model_input['labels'] = labels['input_ids']
    return model_input

tokenized_dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

**Model and Training Setup**

In [19]:
import os
os.environ["WANDB_DISABLED"] = "true"  # disables Weights & Biases tracking

from transformers import TrainingArguments, Trainer

model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = TrainingArguments(
    output_dir="./matchflow_model",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=6,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-19-19df388a790b>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

**Test Generation Cell**

In [ ]:
def generate_recap(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    output = model.generate(**inputs, max_length=300)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example use
test_prompt = "Recap this game: Match: Vassar College Women 34 - 12 Yale University Women. Tries: Bailey (2), Dourmashkin (1), Robinson (1). Conversions: Lynch (2). Cards: None. Injuries: None. Notable: Vassar dominated possession and finished strong with late tries from Bailey and Robinson."
print(generate_recap(test_prompt))